In [1]:
from IPython.display import Markdown as md
import datetime
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass

/home/lukasz/Devel/Covid19


In [2]:
# Configure opentelemetry. This has to be done before we import anything we want to trace

from opentelemetry import trace
from opentelemetry.exporter import jaeger
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchExportSpanProcessor
from opentelemetry.instrumentation.requests import RequestsInstrumentor

RequestsInstrumentor().instrument()
trace.set_tracer_provider(TracerProvider())
jaeger_exporter = jaeger.JaegerSpanExporter(
    service_name="COVID-19-notebook",
    agent_host_name="localhost",
    agent_port=6831,
)
span_processor = BatchExportSpanProcessor(jaeger_exporter)
trace.get_tracer_provider().add_span_processor(span_processor)
tracer = trace.get_tracer(__name__)

# Poland daily data

In [3]:
import figures
import poland_stats as pl
import countries


population = countries.load_countries_data()
pl_population = population['Poland']
NO_OF_DAYS = 90

df = pl.covid_19_timeline()



In [11]:
import plotly.graph_objects as go


def population_percentage_chart(df):
    df["Quarantined percentage"] = df[pl.QUARANTINED].apply(lambda x: x*100/pl_population)
    df["Active percentage"] = df[pl.ACTIVE].apply(lambda x: x*100/pl_population)
    
    fig = go.Figure()

    fig.add_traces(
        go.Scatter(
            x=df["Date"],
            y=df["Quarantined percentage"],
            mode="lines+markers",
            name="Quarantined",
        )
    )
        
    fig.add_traces(
        go.Bar(
            x=df["Date"],
            y=df["Active percentage"],
            name="Active",
        )
    )
    fig.update_layout(
        title='Poland active cases [%] of population',
        legend_title="Legend",
        yaxis_title="[%] of total population"
    )
    return fig

population_percentage_chart(df).show()

In [5]:
with tracer.start_as_current_span(f'Poland daily'):
    figures.cases_and_active(df, NO_OF_DAYS, 'Poland cases daily and active').show()
    figures.deaths_and_recovered(df, NO_OF_DAYS, 'Poland deaths daily and recovered').show()
    figures.quarantined(df, NO_OF_DAYS, 'Poland quarantined').show()

# Poland and few neighbors weekly

We may combine cases and deaths rates with testing against covid-19 and hospital admission rates if we aggregate data on a weekly basis

In [13]:
import ecdc
import trends

country_pop = {'Poland': population['Poland'],
               'Germany': population['Germany'],
               'Czechia': population['Czechia'],
               'Slovakia': population['Slovakia']}


countries_data = []

with tracer.start_as_current_span(f'Countries weekly'):
    cases = ecdc.cases_by_country()
    testing = ecdc.testing_by_country_weekly()
    hospital_rates = ecdc.hospitel_admission_rates_weekly()
    for c in country_pop.keys():
        data = trends.weekly(cases, c, testing, hospital_rates)
        countries_data.append(data)#(data[0:-1]) to drop current week
        
        country_labels = [f'{k} ({round(v/10**6)}M)' for k,v in country_pop.items()]
        
    figures.case_distribution_subplots(countries_data, country_labels, "COVID-19 weekly").show()